<a href="https://colab.research.google.com/github/Kira1108/PromptEngineering/blob/main/Prompt_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**你需要做的就是非常耐心的，特别耐心的写好自己的Prompt**     
**写好以后，你就只需要用到这个prompt的format方法就够了**    
**这个format方法接受你提供的内容文本， 作为参数 `kwargs`**。  

In [6]:
from IPython.display import clear_output
!pip install openai langchain sentencepiece tiktoken
clear_output()

### The Prompt Template Class

```python
class BasePromptTemplate:

    # (Factory method) use this method to create a prompt template
    @classmethod
    def from_template(cls, str) -> BasePromptTemplate:
        ...

    # use this method to return a prompt value(usually inside langchian)
    def format_prompt(self, **input_variables) -> PromptValue:
        ...

    # get a string prompt and return as string for llms
    def format(self, **input_variables) -> string:
        ...

class PromptValue:
    """Finally llms only understand string input or messages input(chat models)"""
    def to_string(self) -> str:
        ...

    def to_messages(self) -> list[Messages]:
        ...
```

In [8]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

In [38]:
# in this prompt template,the input variables ['story', 'emotion'] is automatically parsed
prompt = PromptTemplate.from_template("Tell me a story about {topic}, with emotion {emotion}")
print(prompt.input_variables) 
print(prompt.format(topic = "Dog", emotion = 'angry'))

# use chat prompt the same way as promptTempate, except that this prompt is frmo the user role
chat_prompt = ChatPromptTemplate.from_template("Tell me a story about a {topic} with emotion {emotion}")
print(chat_prompt.input_variables)
print(chat_prompt.format(topic = "Dog", emotion ='angry'))

['emotion', 'topic']
Tell me a story about Dog, with emotion angry
['topic', 'emotion']
Human: Tell me a story about a Dog with emotion angry


In [39]:
# in this prompt template,the input variables ['story', 'emotion'] is automatically parsed
prompt = PromptTemplate.from_template("Tell me a story about {topic}, with emotion {emotion}")
print("Prompt Template")
print(prompt.input_variables)
print(prompt.format_prompt(topic = "Dog", emotion = 'angry'))

print("-"*20)

print("Chat Prompt Template")
# use chat prompt the same way as promptTempate, except that this prompt is frmo the user role
chat_prompt = ChatPromptTemplate.from_template("Tell me a story about a {topic} with emotion {emotion}")
print(chat_prompt.input_variables)
print(chat_prompt.format_prompt(topic = "Dog", emotion ='angry'))

Prompt Template
['emotion', 'topic']
text='Tell me a story about Dog, with emotion angry'
--------------------
Chat Prompt Template
['topic', 'emotion']
messages=[HumanMessage(content='Tell me a story about a Dog with emotion angry', additional_kwargs={}, example=False)]


Converting to string or messages

In [46]:
to_string_result = prompt.format_prompt(topic = "Dog", emotion = 'angry').to_string()
to_messages_result = prompt.format_prompt(topic = "Dog", emotion = 'angry').to_messages()
print(type(to_string_result),": ", to_string_result)
print(type(to_messages_result),": ", to_messages_result)

<class 'str'> :  Tell me a story about Dog, with emotion angry
<class 'list'> :  [HumanMessage(content='Tell me a story about Dog, with emotion angry', additional_kwargs={}, example=False)]


In [47]:
to_string_result = chat_prompt.format_prompt(topic = "Dog", emotion = 'angry').to_string()
to_messages_result = chat_prompt.format_prompt(topic = "Dog", emotion = 'angry').to_messages()
print(type(to_string_result),": ", to_string_result)
print(type(to_messages_result),": ", to_messages_result)

<class 'str'> :  Human: Tell me a story about a Dog with emotion angry
<class 'list'> :  [HumanMessage(content='Tell me a story about a Dog with emotion angry', additional_kwargs={}, example=False)]


## Practice


A PromptTemplate requires 2 arguments `template` and `input_variables`

In [62]:
from langchain import PromptTemplate

template = "What is the fuking wrong with {person}"

# use the constructor
pt = PromptTemplate(template = template, input_variables = ['person'])
wanghuan_prompt = pt.format(person = "Wang Huan")
print(wanghuan_prompt)
print(pt.format(person = "Your Mother!"))
print(pt.format(person = "Your Daddy!"))
print(pt.format(person = "Your Grandpa"))

# or simple use this factory method
fucking_girl_friend = PromptTemplate.from_template(template).format(person = "my girl friend")
print(fucking_girl_friend)

What is the fuking wrong with Wang Huan
What is the fuking wrong with Your Mother!
What is the fuking wrong with Your Daddy!
What is the fuking wrong with Your Grandpa
What is the fuking wrong with my girl friend


## Love Jinja `{{}}`more than Python f-string

In [66]:
jinja_str = "Hi {{person}}, what's the fuking wrong with {{other_person}}."

jinja_template = PromptTemplate.from_template(jinja_str, template_format = 'jinja2')
jinja_template.format(person = "Huan", other_person = "your wife")

"Hi Huan, what's the fuking wrong with your wife."

## Save and load prompts

In [69]:
jinja_template.save("jp.json")
!cat jp.json

{
    "input_variables": [
        "other_person",
        "person"
    ],
    "output_parser": null,
    "partial_variables": {},
    "template": "Hi {{person}}, what's the fuking wrong with {{other_person}}.",
    "template_format": "jinja2",
    "validate_template": true,
    "_type": "prompt"
}

In [70]:
pt.save("pt.json")
!cat pt.json

{
    "input_variables": [
        "person"
    ],
    "output_parser": null,
    "partial_variables": {},
    "template": "What is the fuking wrong with {person}",
    "template_format": "f-string",
    "validate_template": true,
    "_type": "prompt"
}

In [71]:
from langchain.prompts import load_prompt

In [74]:
load_prompt("pt.json").format(person = "wanghuan")

'What is the fuking wrong with wanghuan'

In [75]:
load_prompt("jp.json").format(person = 'wanghuan',other_person = "my wife")

"Hi wanghuan, what's the fuking wrong with my wife."